In [8]:
import bz2
import json
from sklearn.model_selection import train_test_split 

In [75]:
def readFeaturesWithSums(jsonTest,test):
    res=[]
    #res.append(['match_id','r_xp_sum','d_xp_sum','r_gold_sum','d_gold_sum','r_lh_sum',
    #            'd_lh_sum','d_killers_count','r_killers_count'])
    res.append(['match_id','r_xp_sum','d_xp_sum','r_gold_sum','d_gold_sum'])
    with bz2.BZ2File(jsonTest) as matches_file:
        for line in matches_file:
            match = json.loads(line)
            r_xp_sum=0
            d_xp_sum=0
            r_gold_sum=0
            d_gold_sum=0
            r_lh_sum=0
            d_lh_sum=0
            r_killers_count=len(match['players'][0]['kills_log'])
            +len(match['players'][1]['kills_log'])
            +len(match['players'][2]['kills_log'])
            +len(match['players'][3]['kills_log'])
            +len(match['players'][4]['kills_log'])
            d_killers_count=len(match['players'][5]['kills_log'])
            +len(match['players'][6]['kills_log'])
            +len(match['players'][7]['kills_log'])
            +len(match['players'][8]['kills_log'])
            +len(match['players'][9]['kills_log'])
            
            for i in range(len(match['times'])):
                if(match['times'][i]<=5*60):
                    r_xp_sum=match['players'][0]['xp_t'][i]+match['players'][1]['xp_t'][i]+match['players'][2]['xp_t'][i]+match['players'][3]['xp_t'][i]+match['players'][4]['xp_t'][i]
        
                    d_xp_sum=match['players'][5]['xp_t'][i]\
                    +match['players'][6]['xp_t'][i]\
                    +match['players'][7]['xp_t'][i]\
                    +match['players'][8]['xp_t'][i]\
                    +match['players'][9]['xp_t'][i]
        
                    r_gold_sum=match['players'][0]['gold_t'][i]\
                    +match['players'][1]['gold_t'][i]\
                    +match['players'][2]['gold_t'][i]\
                    +match['players'][3]['gold_t'][i]\
                    +match['players'][4]['gold_t'][i]
        
                    d_gold_sum=match['players'][5]['gold_t'][i]\
                    +match['players'][6]['gold_t'][i]\
                    +match['players'][7]['gold_t'][i]\
                    +match['players'][8]['gold_t'][i]\
                    +match['players'][9]['gold_t'][i]
     
                    r_lh_sum=match['players'][0]['lh_t'][i]\
                    +match['players'][1]['lh_t'][i]\
                    +match['players'][2]['lh_t'][i]\
                    +match['players'][3]['lh_t'][i]\
                    +match['players'][4]['lh_t'][i]
        
                    d_lh_sum=match['players'][5]['lh_t'][i]\
                    +match['players'][6]['lh_t'][i]\
                    +match['players'][7]['lh_t'][i]\
                    +match['players'][8]['lh_t'][i]\
                    +match['players'][9]['lh_t'][i]
            #res.append([match['match_id'],r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum,
            #            r_lh_sum,d_lh_sum,d_killers_count,d_killers_count])
            res.append([match['match_id'],r_xp_sum,d_xp_sum,r_gold_sum,d_gold_sum])
    import csv
    with open('tmp.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerows(res)
    features = pandas.read_csv(test)    
    s = pandas.read_csv('./tmp.csv', index_col='match_id')
    result = pandas.merge(features, s, left_on='match_id', right_index=True, how='left', sort=False)
    return result

In [86]:
'''
#test=readFeaturesWithSums('matches_test.jsonlines.bz2','features_test.csv')
#print('test')
features=readFeaturesWithSums('matches.jsonlines.bz2','features.csv')
#features=pandas.read_csv('features.csv')
#test=pandas.read_csv('features_test.csv')
print('features')
features.fillna(9999999,inplace=True)
test.fillna(9999999,inplace=True)
y=features['radiant_win']
features=features.drop(['duration','radiant_win','tower_status_radiant',
               'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)

'''
#features=readFeaturesWithSums('matches.jsonlines.bz2','features.csv')
#print('reading\'ve done')
#features.fillna(9999999,inplace=True)
#y=features['radiant_win']
#features=features.drop(['duration','radiant_win','tower_status_radiant',
#               'tower_status_dire','barracks_status_radiant','barracks_status_dire'],axis=1)
clf=GradientBoostingClassifier(n_estimators=500, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(features, y,test_size=0.2,random_state=42)
#clf.fit(X_train,y_train)
#pred=clf.predict_proba(X_test)[:,1]
#print(roc_auc_score(y_test,pred))
clf.fit(features,y)
print('fit, yea')
test=readFeaturesWithSums('matches_test.jsonlines.bz2','features_test.csv')
test.fillna(9999999,inplace=True)
pred=clf.predict_proba(test)[:,1]
match_id=pandas.read_csv('./features_test.csv')['match_id']
total=[]
total.append(["match_id","radiant_win"])
for i in range(0,len(match_id)):
    total.append([match_id[i],pred[i]])
import csv
with open('res.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(total)
('done')

fit, yea


'done'

In [85]:
features.shape

(97230, 107)

In [44]:
pred=clf.predict_proba(test)[:,1]
match_id=pandas.read_csv('./features_test.csv')['match_id']
total=[]
total.append(["match_id","radiant_win"])
for i in range(0,len(match_id)):
    total.append([match_id[i],pred[i]])
import csv
with open('res.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(total)

In [18]:
#Repeat good models
d_features=features
d_y=y
d_test=test
d_clf=GradientBoostingClassifier(n_estimators=180, random_state=42,max_depth=7)
d_clf.fit(d_features,d_y)
d_pred=d_clf.predict_proba(d_test)[:,1]
d_match_id=pandas.read_csv('./features_test.csv')['match_id']
d_total=[]
d_total.append(["match_id","radiant_win"])
for i in range(0,len(d_match_id)):
    d_total.append([d_match_id[i],d_pred[i]])
import csv
with open('d_res.csv', 'w', newline='') as csvfile:
    d_spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    d_spamwriter.writerows(d_total)

In [22]:
y.iloc[y_test]

1    1
1    1
1    1
0    1
0    1
1    1
1    1
1    1
1    1
1    1
0    1
1    1
0    1
0    1
0    1
1    1
0    1
1    1
0    1
1    1
1    1
1    1
1    1
1    1
0    1
1    1
1    1
1    1
0    1
1    1
    ..
1    1
0    1
0    1
1    1
0    1
1    1
0    1
1    1
1    1
0    1
1    1
1    1
0    1
1    1
1    1
0    1
1    1
0    1
0    1
1    1
1    1
0    1
1    1
1    1
0    1
1    1
0    1
1    1
0    1
1    1
Name: radiant_win, Length: 19446, dtype: int64